In [ ]:
#Figure 2a graph plotting 
#v1.0 version, this is the first consolidation of analysis done so far
#Requires the environment packages found in 23pubenv

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors, cm
from matplotlib.collections import LineCollection
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from tqdm import tqdm

In [ ]:
def read_tuple_list(x):
    x_strip = x.strip('()').strip('[]')
    if len(x_strip.split(', ')) == 1:
        return np.array(x_strip.split(), dtype=float)
    else:
        return np.array(x_strip.split(', '), dtype=float)

In [ ]:
#To consolodate all the wave dataframes in a set, use the following function: 
#INPUT: Sorted list of all wave dataframe csv locations:
#Option to save the DF as a csv in the relevant location is given, default is False
#OUTPUT: A dataframe containing all of the csv wave DFs in the path list

def consolidate_waves(WAVE_DF_PATHS, save_DF = False):
    
    wave_collect = []

    for wave_df_path in tqdm(WAVE_DF_PATHS):
        condition_id = wave_df_path.stem.split('_')[0] #Get the image ID to match to the wave:
        wave_df = pd.read_csv(wave_df_path, converters={"wave_centroid": read_tuple_list}) #Open the wave_DF from step4 analysis
        wave_df = wave_df[wave_df.columns.drop(list(wave_df.filter(regex='Unnamed')))] #Strip it of any weird unnamed index columns
        track_ids = wave_df['clTrackID'].values #Get the collective track values in the wave
        wave_df['wave_ID'] = [str(condition_id) + '-' + str(i) for i in track_ids] #Build a list of unique wave IDs from the image number and clTrack number they belong to
        wave_df['condidtion_ID'] = [condition_id for i in range(len(wave_df))]
        wave_collect.append(wave_df)
    
    wave_df_collected = pd.concat(wave_collect, ignore_index=True)
    
    if save_DF:
        wave_df_collected.to_csv(WAVE_DF_PATHS[0].parent / 'combined_wave_DF.csv', index=False)
        
    return wave_df_collected

In [ ]:
#Lets import an arcos set, wave data, and the corresponding tracking data...
base_path = Path.cwd().parent / Path('Fig2 - Wave kinetics/DF_CSVs')


wave_dataframe_paths = sorted(base_path.glob('*.csv'))


In [ ]:
#Check the base path to confirm:
base_path

In [ ]:
wave_con_df = consolidate_waves(wave_dataframe_paths)

In [ ]:
#Since waves have varying travel distances:
#Here we take the shortest traveling wave, and cut off all longer travel lengths. 
#This ensures that all waves measured are the same length over the averages presented:

cond_col = []

for group_ID, group in wave_con_df.groupby('condidtion_ID'):
    track_lens = []
    for wave_ID, wave in group.groupby('wave_ID'):
        track_lens.append(len(wave))
    min_len = np.min(track_lens)
    cond_col.append(group[group['rel_time'] <= min_len])
    

snipped_wave_df = pd.concat(cond_col, ignore_index=True)

In [ ]:
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2.5, 'font.family': ['arial'], 'axes.linewidth':2})

In [ ]:
plt.figure(figsize=(2,3.5))

ax = sns.lineplot(data=snipped_wave_df,x='real_time', y='r_squared', hue='condidtion_ID', errorbar='sd', palette=["#D27770", sns.color_palette()[0]], legend=False)

plt.xlim(0,min_len*5)
plt.ylim(-1000,45000)
ax.tick_params(direction='out', width=2, labelsize=10)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.xlabel('')
plt.ylabel('')
plt.savefig('fig2-wt-ltb4-dual-plot.png', dpi=400, bbox_inches='tight')

In [ ]:
no_dpi[no_dpi['condidtion_ID'] == 'biil315'].groupby('real_time').mean('rel_r')['rel_r'].loc[0:60].to_csv('LTB4block_early_wave_mean_travel.csv')